# AWS CLI Introduction

This is a jupyter notebook that uses the [bash_kernel](https://pypi.org/project/bash_kernel/) to introduce the AWS CLI. Please complete the steps in the [README](./README.md) before running this notebook.

## AWS CLI

The AWS Command Line Interface (AWS CLI) is an open source tool that enables you to interact with AWS services using commands in your command-line shell. See the [documentation](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-welcome.html) for more details.

Amazon Linux hosts used in most labs will have AWS CLI v1 preinstalled. The ACO course and demo video also describe the v1 of the CLI, however development on v1 has now stopped so new services are now only added to v2. v2 is [mostly compatible](https://docs.aws.amazon.com/cli/latest/userguide/cliv2-migration.html) with v1 and has some nice usability improvements, but the [installation](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html) is a bit more convoluted.

Activity 1 in the ACO course uses a RedHat instance to show students how to install and configure the CLI for their account. If students have their own accounts, or AWS Educate Starter accounts, encourage them to configure the CLI on their desktop to remote access those accounts. You can suggest to your students to use v2 for this activity and for their local installation.

## CLI Installation

Follow the instructions for either [v1](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv1.html) or [v2](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html).

What version do you have installed?

In [ ]:
aws --version

## Authentication

In a SageMaker notebook your credentials will already be set to the role ypu specified when you created the notebook. For your own account you may need to first run `aws configure` command to set the user access and secret access keys to authenticate with your account.

We can use the AWS Security Token Service to confirm we have properly authenticated to our account. If running in a AWS SageMaker notebook you will have the permissions you granted the role you used when creating the notebook. If running in a different environment, for example on your desktop, you may need to provide your user's access and secret access key.

If this returns an error like _"The security token included in the request is invalid."_ you will need to recheck your authentication.

In [3]:
export AWS_ACCESS_KEY_ID=
export AWS_SECRET_ACCESS_KEY=
export AWS_SESSION_TOKEN=
aws sts get-caller-identity

{
    "UserId": "AROAJ2WQIZXYSGGVGF37E:dcchatte-Isengard",
    "Account": "133766463080",
    "Arn": "arn:aws:sts::133766463080:assumed-role/Admin/dcchatte-Isengard"
}


## CLI Help

The aws command format looks like this: 
`aws` `<command>` `<subcommand>` `[options and parameters]`

A `command` could be _ec2_, _s3_, _rds_ or _secretsmanager_, the full list can seen with `aws help`. Possible `subcommand` depend on the `command` selected and these are listed using _help_ as the `subcommand`.

In [4]:
aws sts help

STS()                                                                    STS()



NAME
       sts -

DESCRIPTION
       The  AWS Security Token Service (STS) is a web service that enables you
       to request temporary, limited-privilege credentials  for  AWS  Identity
       and  Access  Management  (IAM) users or for users that you authenticate
       (federated users). This guide provides descriptions of the STS API. For
       more  detailed  information  about  using this service, go to Temporary
       Security Credentials .

       For information about setting up signatures and  authorization  through
       the  API, go to Signing AWS API Requests in the AWS General Reference .
       For general information  about  the  Query  API,  go  to  Making  Query
       Requests  in  Using  IAM  . For information about using security tokens
       with other AWS products, go to AWS Services That Work with IAM  in  the
       IAM User Guide .

       If  you're new to AWS and need add

In [5]:
aws sts get-caller-identity help

GET-CALLER-IDENTITY()                                    GET-CALLER-IDENTITY()



NAME
       get-caller-identity -

DESCRIPTION
       Returns  details  about the IAM user or role whose credentials are used
       to call the operation.

       NOTE:
          No permissions are required to perform this operation. If an  admin-
          istrator  adds  a  policy  to  your IAM user or role that explicitly
          denies access to the sts:GetCallerIdentity  action,  you  can  still
          perform  this  operation.  Permissions  are not required because the
          same information is returned when an IAM  user  or  role  is  denied
          access. To view an example response, see I Am Not Authorized to Per-
          form: iam:DeleteVirtualMFADevice in the IAM User Guide .

       See also: AWS API Documentation

       See 'aws help' for descriptions of global parameters.

SYNOPSIS
            get-caller-identity
          [--cli-input-json | --cli-input-yaml]
          [--ge

## Output, Query and Filters

Every account in each region always has a default VPC so you can always explore VPCs without have to first create a resource to explore. The [describe-vpcs](https://awscli.amazonaws.com/v2/documentation/api/latest/reference/ec2/describe-vpcs.html) subcommand for `ec2` will output a large JSON blob describing all the VPCs in this account in this region.

If you get an error with this command, check what policies are attached to the role you are using and ensure it has full access to EC2 before trying again.

In [6]:
aws ec2 describe-vpcs

{
    "Vpcs": [
        {
            "CidrBlock": "172.16.0.0/16",
            "DhcpOptionsId": "dopt-56317531",
            "State": "available",
            "VpcId": "vpc-082bb71da5e445aa4",
            "OwnerId": "133766463080",
            "InstanceTenancy": "default",
            "CidrBlockAssociationSet": [
                {
                    "AssociationId": "vpc-cidr-assoc-067aaa76496202809",
                    "CidrBlock": "172.16.0.0/16",
                    "CidrBlockState": {
                        "State": "associated"
                    }
                }
            ],
            "IsDefault": false,
            "Tags": [
                {
                    "Key": "AWSServiceAccount",
                    "Value": "531097355252"
                }
            ]
        },
        {
            "CidrBlock": "172.31.0.0/16",
            "DhcpOptionsId": "dopt-56317531",
            "State": "available",
            "VpcId": "vpc-983d67ff",
            "OwnerId": "13

We can [query](https://docs.aws.amazon.com/cli/latest/userguide/cli-usage-output.html#cli-usage-output-filter) a subset of the output using [JMESPath](http://jmespath.org/) strings. From the JSON above we see the VPCs are a list, and within the list we can just pull out specific values like the VpcId and CidrBlock. Notice that we can rename those fields if we want to.

In [7]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}'

[
    {
        "ID": "vpc-082bb71da5e445aa4",
        "CIDR": "172.16.0.0/16",
        "Default": false
    },
    {
        "ID": "vpc-983d67ff",
        "CIDR": "172.31.0.0/16",
        "Default": true
    },
    {
        "ID": "vpc-0edb83a9f67ec506f",
        "CIDR": "10.0.0.0/16",
        "Default": false
    }
]


The output format can be changed with `--output` to `table`, `text` (and with v2 `yaml`):

In [8]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --output table

-------------------------------------------------------
|                    DescribeVpcs                     |
+----------------+----------+-------------------------+
|      CIDR      | Default  |           ID            |
+----------------+----------+-------------------------+
|  172.16.0.0/16 |  False   |  vpc-082bb71da5e445aa4  |
|  172.31.0.0/16 |  True    |  vpc-983d67ff           |
|  10.0.0.0/16   |  False   |  vpc-0edb83a9f67ec506f  |
+----------------+----------+-------------------------+


In [9]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --output text

172.16.0.0/16	False	vpc-082bb71da5e445aa4
172.31.0.0/16	True	vpc-983d67ff
10.0.0.0/16	False	vpc-0edb83a9f67ec506f


In [10]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --output yaml

- CIDR: 172.16.0.0/16
  Default: false
  ID: vpc-082bb71da5e445aa4
- CIDR: 172.31.0.0/16
  Default: true
  ID: vpc-983d67ff
- CIDR: 10.0.0.0/16
  Default: false
  ID: vpc-0edb83a9f67ec506f


If we just want the first VPC returned in the json blob:

In [11]:
aws ec2 describe-vpcs --query 'Vpcs[0].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}'

{
    "ID": "vpc-082bb71da5e445aa4",
    "CIDR": "172.16.0.0/16",
    "Default": false
}


Or we can find the default VPC by using the `?` operator. Note the use of backticks in the query expression.

In [12]:
aws ec2 describe-vpcs --query 'Vpcs[?IsDefault==`true`].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}'

[
    {
        "ID": "vpc-983d67ff",
        "CIDR": "172.31.0.0/16",
        "Default": true
    }
]


That approach is applying the expression to the result returned to the CLI client. The preferred way to specify what resources are returned by a command is use the `--filters` option if a filter exists for your resource.

In [13]:
aws ec2 describe-vpcs --filters 'Name=isDefault,Values=true' --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}'

[
    {
        "ID": "vpc-983d67ff",
        "CIDR": "172.31.0.0/16",
        "Default": true
    }
]


If you know the ID of resource you are looking for you may be able to specify that as a parameter. Replace `VPCID` with the VpcId from the previous output:

In [14]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --vpc-ids VPCID


An error occurred (InvalidVpcID.NotFound) when calling the DescribeVpcs operation: The vpc ID 'vpcid' does not exist


: 254

## Creating Resources

When creating resources there will be some mandatory parameters, as well as optional parameters. From the help for `create-vpc` what parameter is mandatory?

_Note: If using cli v2 in a real shell (not a notebook) you can use the `--cli-auto-prompt` option to be prompted for the mandatory parameters. If autocomplete if setup you can also type the start of the parameter `--` and press tab twice to be shown the list of parameters._


In [17]:
aws ec2 create-vpc help

<string>:162: (WARNING/2) Inline substitution_reference start-string without end-string.
CREATE-VPC()                                                      CREATE-VPC()



NAME
       create-vpc -

DESCRIPTION
       Creates  a VPC with the specified IPv4 CIDR block. The smallest VPC you
       can create uses a /28 netmask (16 IPv4 addresses), and the largest uses
       a  /16  netmask (65,536 IPv4 addresses). For more information about how
       large to make your VPC, see Your VPC and Subnets in the Amazon  Virtual
       Private Cloud User Guide .

       You  can  optionally  request  an  IPv6 CIDR block for the VPC. You can
       request an Amazon-provided IPv6 CIDR block from Amazon's pool  of  IPv6
       addresses,  or  an  IPv6  CIDR block from an IPv6 address pool that you
       provisioned through bring your own IP addresses (BYOIP ).

       By default, each instance you launch in the VPC has  the  default  DHCP
       options,  which include only a default DNS server t

Create a VPC with a CIDR block 10.10.0.0/16 by replacing `COMPLETE_ME` below.

In [32]:
aws ec2 create-vpc COMPLETE_ME

{
    "Vpc": {
        "CidrBlock": "10.10.0.0/16",
        "DhcpOptionsId": "dopt-56317531",
        "State": "pending",
        "VpcId": "vpc-002a8ec8f14a25b3c",
        "OwnerId": "133766463080",
        "InstanceTenancy": "default",
        "Ipv6CidrBlockAssociationSet": [],
        "CidrBlockAssociationSet": [
            {
                "AssociationId": "vpc-cidr-assoc-0c8764ec4873a8c4f",
                "CidrBlock": "10.10.0.0/16",
                "CidrBlockState": {
                    "State": "associated"
                }
            }
        ],
        "IsDefault": false
    }
}


How would we list the set of VPCs that only have a CIDR range starting with 10? This can be done with a filter or a query.

In [19]:
aws ec2 describe-vpcs COMPLETE_ME

[
    {
        "ID": "vpc-0edb83a9f67ec506f",
        "CIDR": "10.0.0.0/16",
        "Default": false
    },
    {
        "ID": "vpc-003467fd272a8bf12",
        "CIDR": "10.10.0.0/16",
        "Default": false
    }
]


To make our VPC more recognisable, add a tag called `Name` to the VPC, replacing `VPCID` with your VPC's ID. Note how tags make the query more complicated.

In [33]:
aws ec2 create-tags \
    --resources VPCID \
    --tags Key=Name,Value="notebook-demo-vpc"

aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault,Name:Tags[?Key == `Name`] | [0].Value}' --output table

----------------------------------------------------------------------------
|                               DescribeVpcs                               |
+---------------+----------+-------------------------+---------------------+
|     CIDR      | Default  |           ID            |        Name         |
+---------------+----------+-------------------------+---------------------+
|  10.10.0.0/16 |  False   |  vpc-002a8ec8f14a25b3c  |  notebook-demo-vpc  |
|  172.16.0.0/16|  False   |  vpc-082bb71da5e445aa4  |  None               |
|  172.31.0.0/16|  True    |  vpc-983d67ff           |  None               |
|  10.0.0.0/16  |  False   |  vpc-0edb83a9f67ec506f  |  Lab VPC            |
+---------------+----------+-------------------------+---------------------+


There are two things annoying about what we just did; cutting and pasting the VPCID and have to add a tag to the resource after creating the resource. The later problem has been fixed in v2 with the introduction of `--tag-specifications` to many commands.

To fix the first problem we should use shell variables to capture the IDs for our resources as we create them. Replace `VPCID` in the following command to put your VPC's ID into the shell variable called `VPC`.

In [23]:
VPC=VPCID

Create a subnet in the new VPC with a CIDR block of 10.10.1.0/24. Note the query is just returning the subnet's ID as a text value.

In [28]:
SUBNET=$(aws ec2 create-subnet \
    --vpc-id $VPC \
    --cidr-block 10.10.1.0/24 \
    --query 'Subnet.SubnetId' \
    --output text)

echo $SUBNET

subnet-0b4b1c3e8c97af409


Some resources take time to create, like EC2 instances, RDS instances and EBS snapshots. The CLI commands will return before they are done, so you may need to use the `wait` command to know when they are operational.

In [29]:
BASE_AMI=$(aws ssm get-parameter \
    --name /aws/service/ami-amazon-linux-latest/amzn2-ami-hvm-x86_64-gp2 \
    --query 'Parameter.Value' \
    --output text)

INSTANCE=$(aws ec2 run-instances \
    --instance-type t2.micro \
    --associate-public-ip-address \
    --image-id $BASE_AMI \
    --subnet-id $SUBNET \
    --output text \
    --query 'Instances[*].InstanceId')

echo $INSTANCE

aws ec2 wait instance-running --instance-ids $INSTANCE 

aws ec2 describe-instances \
    --instance-ids $INSTANCE \
    --output table \
    --query "Reservations[*].Instances[0].{ID:InstanceId,BlockDevices:BlockDeviceMappings[*].{Device:DeviceName,Attached:Ebs.Status,Volume:Ebs.VolumeId},Network:NetworkInterfaces[*].{SecGroups:Groups[*].GroupId,IP:PrivateIpAddresses[*].{PublicIP:Association.PublicIp,PublicDNS:Association.PublicDnsName,PrivateIP:PrivateIpAddress,Primary:Primary},ID:NetworkInterfaceId}}"

i-09cca2d2cd31b6739
-----------------------------------------------------------
|                    DescribeInstances                    |
+---------------------------------------------------------+
|                           ID                            |
+---------------------------------------------------------+
|  i-09cca2d2cd31b6739                                    |
+---------------------------------------------------------+
||                     BlockDevices                      ||
|+------------+--------------+---------------------------+|
||  Attached  |   Device     |          Volume           ||
|+------------+--------------+---------------------------+|
||  attached  |  /dev/xvda   |  vol-08036aa44b90f0746    ||
|+------------+--------------+---------------------------+|
||                        Network                        ||
|+-------------------------------------------------------+|
||                          ID                           ||
|+------------------

## Cleanup
In general you should remove resources in the reverse order they were created in, effectively popping off a stack. We need to wait for the instance to terminate before we can delete the subnet and VPC.

In [30]:
aws ec2 terminate-instances --instance-ids $INSTANCE
aws ec2 wait instance-terminated --instance-ids $INSTANCE
echo "$INSTANCE Terminated"
aws ec2 delete-subnet --subnet-id $SUBNET
aws ec2 delete-vpc --vpc-id $VPC
echo "$VPC Deleted"

{
    "TerminatingInstances": [
        {
            "CurrentState": {
                "Code": 32,
                "Name": "shutting-down"
            },
            "InstanceId": "i-09cca2d2cd31b6739",
            "PreviousState": {
                "Code": 16,
                "Name": "running"
            }
        }
    ]
}


In [31]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --output table

-------------------------------------------------------
|                    DescribeVpcs                     |
+----------------+----------+-------------------------+
|      CIDR      | Default  |           ID            |
+----------------+----------+-------------------------+
|  172.16.0.0/16 |  False   |  vpc-082bb71da5e445aa4  |
|  172.31.0.0/16 |  True    |  vpc-983d67ff           |
|  10.0.0.0/16   |  False   |  vpc-0edb83a9f67ec506f  |
+----------------+----------+-------------------------+
